To do:
- ADD WEIGHTED LOSS FUNCTION FOR HITS ON 1 FOR POSITIVE LABELLING AS THIS IS WAY MORE RARE = HIGHER "REWARD"
- Have a look at learning rate and gradient norm clipping which I need to read up on.
    - In addition to this, look at the implications of gradient accumulation steps
    - Much of this pipeline was constrained due to computational restrictions which I think was caused by errors and not actual training process.
- Early stopping: load_best_model_at_end=True
- Split into validation set as well (80:10:10)
- Hyperparameter tuning (Alpha, learning rate, batch size so on - not sure how to figure this out)
    - There is precedence for no hyperparameter tuning from the author of the OG NLI model that DEBATE is based on = Due to computational restrains and the points from this paper, no hyperparameter tuning was performed in this case. The model tuning in itself is also not the primary focus in this paper, but simply serves as a tool for the actual inquiry into blame in the Danish Parliament
- Context / No context?
    - Most likely no context, as we do not have the academic reasons for doing so 
    - Model actually also seems to perform better without (f1 of .66 for no context on random test set vs. . 55 on gold labelled for context model) This might also make sense as the context is not taken into account in the DEBATE model


In [7]:
%pip install -r "requirements_bert.txt"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 94.3 MB/s  0:00:04m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 24.8 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 24.6 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 32.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 71.7 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15/15 [tensorflow]5 [tensorflow]]]ata-server]
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
import transformers
from peft import LoraConfig, get_peft_model, TaskType
import bitsandbytes
import accelerate
import datasets
#import scikit-learn
import numpy as np
import pandas as pd
from datetime import datetime
from transformers import Conv1D, AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig, AutoModelForCausalLM
from sklearn.model_selection import train_test_split
from datasets import Dataset

/work/RuneEgeskovTrust#9638/miniconda3/envs/blame/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "jhu-clsp/mmBERT-base"

quantization_config = BitsAndBytesConfig(
                                        load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.bfloat16,
                                         bnb_4bit_quant_type="nf4",
                                         bnb_4bit_use_double_quant=True,
                                         )

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quantization_config,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

  2025-10-20T08:02:24.026013Z  WARN  Status Code: 500. Retrying..., request_id: "01K80A84K4X6QCYH5WJ470PWDZ"
    at /home/runner/work/xet-core/xet-core/cas_client/src/http_client.rs:227

  2025-10-20T08:02:24.026059Z  WARN  Retry attempt #0. Sleeping 414.947056ms before the next attempt
    at /root/.cargo/registry/src/index.crates.io-1949cf8c6b5b557f/reqwest-retry-0.7.0/src/middleware.rs:171

  2025-10-20T08:02:24.548261Z  WARN  Status Code: 500. Retrying..., request_id: "01K80A853FAN03F6GCFPDN6R4G"
    at /home/runner/work/xet-core/xet-core/cas_client/src/http_client.rs:227

  2025-10-20T08:02:24.548280Z  WARN  Retry attempt #1. Sleeping 3.561067537s before the next attempt
    at /root/.cargo/registry/src/index.crates.io-1949cf8c6b5b557f/reqwest-retry-0.7.0/src/middleware.rs:171

  2025-10-20T08:02:28.216110Z  WARN  Status Code: 500. Retrying..., request_id: "01K80A88P20ZPY6EN11D1J3B0Q"
    at /home/runner/work/xet-core/xet-core/cas_client/src/http_client.rs:227

  2025-10-20T08:02:

OSError: Can't load the model for 'jhu-clsp/mmBERT-base'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'jhu-clsp/mmBERT-base' is the correct path to a directory containing a file named pytorch_model.bin, tf_model.h5, model.ckpt or flax_model.msgpack.

In [3]:
lora_config = LoraConfig(
    r=8,  # Low-rank dimension
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["Wqkv"],  # Fine-tuning the attention layer specifically
)

lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters()

NameError: name 'model' is not defined

In [ ]:
'''
Consider the batch size, could be increased for efficiency purposes.
ADD WEIGHTED LOSS FUNCTION FOR HITS ON 1 FOR POSITIVE LABELLING AS THIS IS WAY MORE RARE = HIGHER "REWARD"
Have a look at learning rate and gradient norm clipping which I need to read up on.
    In addition to this, look at the implications of gradient accumulation steps
    Much of this pipeline was constrained due to computational restrictions which I think was caused by errors and not actual training process.
Early stopping: load_best_model_at_end=True
'''

training_args = TrainingArguments(
    output_dir='./full_results',
    learning_rate=8e-4, # Learning rate copied from mmBERT paper as they found this to perform best
    num_train_epochs=3,
    per_device_train_batch_size=8,
    auto_find_batch_size=True, # Allows for auto adjusting of batch to avoid OOM
    gradient_accumulation_steps=12,  # Simulate larger batch size

    logging_steps=1,
    eval_strategy="epoch",
    save_strategy="epoch",
    fp16=True,  # Enable mixed precision
    dataloader_pin_memory=False,
    remove_unused_columns=True, # Avoiding manual handling of residual text columns
    max_grad_norm=1.0,

    disable_tqdm=False,
)

In [ ]:
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, average_precision_score


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        'precision:' average_precision_score(labels, predictions),
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions, average='macro') # Macro is better suited for imbalanced data
    }

In [ ]:

dataframe = pd.read_json("/work/RuneEgeskovTrust#9638/Bachelor/training_data/cleaned_training_data_3_4_5_temps.json")

val_dataframe = pd.read_json("/work/RuneEgeskovTrust#9638/Bachelor/Bachelor_project/Model_data/validation_set.json")


In [8]:
val_dataframe = val_dataframe[['preceding_sentence', 'text', 'succeeding_sent', 'label']]

dataframe = dataframe[['preceding_sentence', 'text', 'succeeding_sent', 'label']]

val_dataset = Dataset.from_pandas(val_dataframe)

dataset = Dataset.from_pandas(dataframe)

tokenized_val = val_dataset.map(tokenize_function)

tokenized_dataset = dataset.map(tokenize_function)

Map: 100%|██████████| 3600/3600 [00:06<00:00, 543.63 examples/s]


In [ ]:
'''
This has still not been tested, but will be implemented soon

Instead of cross entropy = BCEWITHLOGITSLOSS? https://docs.pytorch.org/docs/1.7.1/generated/torch.nn.BCEWithLogitsLoss.html#torch.nn.BCEWithLogitsLoss 

No matter what, we need to settle on a loss function which allows for weighted loss due to the strong imbalance in the dataset. 

Binary cross-entropy (log loss) is a loss function used in binary classification problems. It quantifies the difference between the actual class labels (0 or 1) and the predicted probabilities output by the model. The lower the binary cross-entropy value, the better the model’s predictions align with the true labels.

Since the model’s output is a probability between 0 and 1, minimizing binary cross-entropy during training helps improve predictive accuracy, ensuring the model effectively distinguishes between two classes.

'''

from collections import Counter

labels = random_dataset['label'].tolist()
class_counts = Counter(labels)
total = sum(class_counts.values())

# Higher weight = more emphasis
weights = [total/class_counts[0], total/class_counts[1]]
class_weights = torch.tensor(weights, dtype=torch.float)

#define custom trainer that uses weigted loss
import torch.nn as nn

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        
        # Define weighted loss
        loss_fct = nn.CrossEntropyLoss(weight=class_weights.to(model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        
        return (loss, outputs) if return_outputs else loss

In [8]:
# An option for BCE (Binary Cross Entropy implementation), Keras definitely the easy way out - they are essentially the same as manual calc and more efficient.
import numpy as np
from keras.losses import binary_crossentropy

# Example true labels and predicted probabilities
y_true = np.array([0, 1, 1, 0, 1])
y_pred = np.array([0.1, 0.9, 0.8, 0.2, 0.7])

# Compute Binary Cross-Entropy using NumPy
def binary_cross_entropy(y_true, y_pred):
    bce = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    return bce

bce_loss = binary_cross_entropy(y_true, y_pred)
print(f"Binary Cross-Entropy Loss (manual calculation): {bce_loss}")

# Compute Binary Cross-Entropy using Keras
bce_loss_keras = binary_crossentropy(y_true, y_pred).numpy()
print(f"Binary Cross-Entropy Loss (Keras): {bce_loss_keras}")

2025-10-20 10:37:54.408384: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-20 10:37:54.463559: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-20 10:37:56.034189: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Binary Cross-Entropy Loss (manual calculation): 0.20273661557656092
Binary Cross-Entropy Loss (Keras): 0.20273661557656092


W0000 00:00:1760949476.398954    2662 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [ ]:
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
)


trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,4.420500,0.508216,0.767045,0.732810
2,2.795000,0.497293,0.812500,0.803830
3,1.363200,0.504073,0.812500,0.800597


TrainOutput(global_step=114, training_loss=5.090314436377141, metrics={'train_runtime': 4870.4503, 'train_samples_per_second': 2.217, 'train_steps_per_second': 0.023, 'total_flos': 5.916996698112e+16, 'train_loss': 5.090314436377141, 'epoch': 3.0})

In [11]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 0.5040732026100159, 'eval_accuracy': 0.8125, 'eval_f1': 0.8005973838706355, 'eval_runtime': 32.1592, 'eval_samples_per_second': 5.473, 'eval_steps_per_second': 0.684, 'epoch': 3.0}


In [13]:

with open("/work/RuneEgeskovTrust#9638/Bachelor/Bachelor_project/Evalresult5templates.txt", "w") as f:
    f.write(str(eval_results))

In [ ]:
'''
We would expect to see a gradual decrease in both training and validation loss.
If either om them split too far from eachother that indicates issues with the training process.
The process itself should be pretty smooth with no dips either up or down.
'''

In [ ]:
# This is where we should very much remember to save the finetuned model locally as this contains the new weights for use in analyzing new text
lora_model.save_pretrained(f"output/mmBERT/full_model")